In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functions import *
    

In [13]:
df = pd.read_csv('factor_trading_1.csv')
df.drop(['Unnamed: 0', 'index', 'long return', 'short return', 'AskPrice2', 'BidPrice2', 'R_square_short', 'R_square_long', 'predict_long_sign', 'predict_short_sign'], axis=1, inplace=True)
#preprocess the data
new_df = df.copy()
new_df['AskPrice1'] = (new_df['AskPrice1'] / 700)
new_df['BidPrice1'] = (new_df['BidPrice1'] / 700)
new_df['predict_short'] = new_df['predict_short'] * 1000
new_df['predict_long'] = new_df['predict_long'] * 1000
# change the name of the columns
new_df.columns = ['ask_price', 'bid_price', 'predicted_return_50', 'predicted_return_100']
new_df

,ask_price,bid_price,predicted_return_50,predicted_return_100
0,1.103571,1.102857,0.778658,-0.318226
1,1.103571,1.102857,0.779251,-0.217621
2,1.103571,1.102857,0.875122,-0.183277
3,1.103571,1.102857,0.786265,-0.252589
4,1.103571,1.102857,0.562608,-0.231198
...,...,...,...,...
73845,1.095000,1.094286,0.086524,0.105890
73846,1.095000,1.094286,0.086361,0.077719
73847,1.095000,1.094286,0.048089,0.074122
73848,1.095000,1.094286,-0.309119,-0.130371


In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Define the policy network
class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(5, 64)  # Increased input dimension to 5
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.softmax(self.fc3(x), dim=1)

# Initialize the neural network and optimizer
policy_net = PolicyNet()
optimizer = optim.Adam(policy_net.parameters(), lr=0.001)

# Sample training data (replace this with your actual training data)
train_data = pd.DataFrame({
    'bid': np.random.rand(3000),
    'ask': np.random.rand(3000),
    'prediction_50': np.random.rand(3000),
    'prediction_100': np.random.rand(3000)
})

# Training Loop
epochs = 20
for epoch in range(epochs):
    capital = 1000  # Initialize capital for each epoch
    holding_position = 0  # Initialize holding position

    for index, row in train_data.iterrows():
        state = torch.tensor([[row['bid'], row['ask'], row['prediction_50'], row['prediction_100'], holding_position]], dtype=torch.float32)
        action_prob = policy_net(state)
        
        action = torch.argmax(action_prob, dim=1).item()  # Choose action (0: Buy, 1: Sell, 2: Hold)
        
        # Simulate capital change and update holding position
        if action == 0 and holding_position < 10:  # Buy
            capital -= row['ask']
            holding_position += 1
        elif action == 1 and holding_position > -10:  # Sell
            capital += row['bid']
            holding_position -= 1

        # Calculate reward based on capital change
        reward = capital - 1000  # Assuming initial capital is 1000
        
        # Compute loss and perform backpropagation
        loss = -torch.log(action_prob[0][action]) * reward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Final capital: {capital}, Final holding position: {holding_position}")





Epoch 1, Final capital: 1004.1848838832196, Final holding position: -10
Epoch 2, Final capital: 1004.1848838832196, Final holding position: -10
Epoch 3, Final capital: 1004.1848838832196, Final holding position: -10
Epoch 4, Final capital: 1004.1848838832196, Final holding position: -10
Epoch 5, Final capital: 1004.1848838832196, Final holding position: -10


KeyboardInterrupt: 

In [ ]:
capital = 1000
for index, row in test_data.iterrows():
    state = torch.tensor([[row['bid'], row['ask'], row['prediction_50'], row['prediction_100']]], dtype=torch.float32)
    action_prob = policy_net(state)
    action = torch.argmax(action_prob, dim=1).item()
    
    if action == 0:
        capital -= row['ask']
    elif action == 1:
        capital += row['bid']